In [27]:
from os import listdir
import pandas as pd
import numpy as np

In [9]:
%%writefile callCoverageMatrix.sh
ls 5.coverageFiles.bacteria/*.coverage | while read -r file
do
    qsub -v FOO=$file runCoverageMatrix.sh
done

Overwriting callCoverageMatrix.sh


In [10]:
%%writefile runCoverageMatrix.sh
#!/bin/bash
#$ -N cov2mat
#$ -l h_vmem=4G
#$ -l h_rt=01:00:00
#$ -cwd

export PATH=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/anaconda3/bin:$PATH
source activate py2_leo

file=$FOO
python ../../code/coverageMatrixFromSamCoverage.py $file 0.bacterialGenomes/BacterialGenomes.length.txt 100000 "$file.matrix.csv"

Overwriting runCoverageMatrix.sh


In [11]:
%%bash
chmod +x callCoverageMatrix.sh
./callCoverageMatrix.sh

Your job 8551309 ("cov2mat") has been submitted
Your job 8551310 ("cov2mat") has been submitted
Your job 8551311 ("cov2mat") has been submitted
Your job 8551312 ("cov2mat") has been submitted
Your job 8551313 ("cov2mat") has been submitted
Your job 8551314 ("cov2mat") has been submitted
Your job 8551315 ("cov2mat") has been submitted
Your job 8551316 ("cov2mat") has been submitted
Your job 8551317 ("cov2mat") has been submitted
Your job 8551318 ("cov2mat") has been submitted
Your job 8551319 ("cov2mat") has been submitted
Your job 8551320 ("cov2mat") has been submitted
Your job 8551321 ("cov2mat") has been submitted
Your job 8551322 ("cov2mat") has been submitted
Your job 8551323 ("cov2mat") has been submitted
Your job 8551324 ("cov2mat") has been submitted
Your job 8551325 ("cov2mat") has been submitted
Your job 8551326 ("cov2mat") has been submitted
Your job 8551327 ("cov2mat") has been submitted
Your job 8551328 ("cov2mat") has been submitted
Your job 8551329 ("cov2mat") has been su

In [17]:
%%bash
mkdir 6.coverageMatrices.bacteria
mv 5.coverageFiles.bacteria/*.matrix.csv 6.coverageMatrices.bacteria

In [19]:
samples = set()
for fileName in listdir("6.coverageMatrices.bacteria/"):
    samples.add(fileName.split('.')[0])
fw = open("catCoverageMatrices.sh","w")
for sample in samples:
    fw.write("cat 6.coverageMatrices.bacteria/"+sample+"* > "+sample+".all.coverage.matrix.csv\n")
fw.close()

In [82]:
%%bash
chmod +x catCoverageMatrices.sh
./catCoverageMatrices.sh

In [83]:
#Read lengths file
referenceLengths =  pd.read_csv("0.bacterialGenomes/BacterialGenomes.length.txt",sep='\t',header=None)
referenceLengths.columns = ["REFERENCE","LENGTH"]
referenceLengths.set_index("REFERENCE",inplace=True)
referenceLengths.sort_values(by="LENGTH",inplace=True)
referenceLengths.head()

,LENGTH
REFERENCE,
MECZ01000002.1,21881
CP000452.1,55635
CM003440.1,61247
CP017830.1,91776
CP013211.1,112031


In [84]:
for matrix in listdir('.'):
    if "coverage.matrix.csv" in matrix:
        covMatrix = pd.read_csv(matrix,header=0,index_col=0,sep='\t')
        covMatrix = covMatrix.loc[np.logical_not(pd.isnull(covMatrix.index))]
        referenceLengthsSample = referenceLengths.loc[covMatrix.index]
        referenceLengthsSample.sort_values(by="LENGTH",inplace=True)
        covMatrix = covMatrix.loc[referenceLengthsSample.index]
        covMatrix.to_csv(matrix,sep='\t',header=True,index=True)

In [87]:
%%bash
mkdir 7.coverageMatrices.sorted.bacteria
mv *.coverage.matrix.csv 7.coverageMatrices.sorted.bacteria

### Finally, run "coverageHeatmap" notebook. It makes the heatmap for all matrix files in folder 7